In [1]:
from pathlib import Path
from openai import OpenAI
import os
from dotenv import load_dotenv
import json

dotenv_path = Path("__file__").resolve().parents[1].parents[0] / '.local.env'
load_dotenv(dotenv_path)

OpenAI.api_key = os.getenv("OPENAI_API_KEY")
data_path = Path("__file__").resolve().parents[1].parents[0] / "local_data"

In [5]:
def is_file_available(file_path):
  return os.path.isfile(file_path) and file_path.endswith('.jsonl')

def is_apikey_valid(api_key_str):
  return bool(api_key_str and api_key_str.strip() and api_key_str.startswith('sk-'))

def validate_training_data(training_data_path):
    with open(training_data_path, 'r') as file:
        lines = file.readlines()

    for index, line in enumerate(lines, start=1):
        data = json.loads(line.strip())
        if not isinstance(data, dict):
            raise ValueError(f'行 {index}: エラー - データは辞書型であるべきです')
        if 'messages' not in data or not isinstance(data['messages'], list):
            raise ValueError(f'行 {index}: エラー - メッセージキーが存在しないか、リスト型でない')
        if len(data['messages']) < 1:
            raise ValueError(f'行 {index}: エラー - メッセージ配列は少なくとも1つの要素を含むべきです')
            # check the first message is system prompt
        first_message = data['messages'][0]
        if first_message.get('role') != 'system':
            raise ValueError(f'行 {index}: エラー - 最初のメッセージはsystemプロンプトであるべきです')

def get_model_name_by_id(ft_id):
  response = OpenAI.FineTuningJob.list(limit=10)
  for mod in response['data']:
    if mod['id'] == ft_id:
      return mod['fine_tuned_model']
  return ""

def get_system_prompt_from_training_data(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            data = json.loads(line)
            assert data['messages'][0]['role'] == 'system'
            return data['messages'][0]['content']

In [7]:
client = OpenAI()

TRAINING_FILE_PATH = "../../local_data/train_text.jsonl"
# 学習データファイルパス（jsonl）

training_data_id = ""

if not is_file_available(TRAINING_FILE_PATH):
  raise ValueError("TRAINING_FILE_PATHは存在しません")

validate_training_data(TRAINING_FILE_PATH)

# upload traning data into openai files
response = client.files.create(file=open(TRAINING_FILE_PATH), purpose='fine-tune')

training_data_id = response['id']

print(f"学習データファイルID: {training_data_id}")
print("passed")

TypeError: Multipart file uploads must be opened in binary mode, not text mode.

In [3]:
openai.File.list()

In [ ]:
response = openai.FineTuningJob.create(training_file=training_data_id, model="gpt-3.5-turbo")
finetuning_id = response['id']
print("ファインチューニングを開始しました。")
print(f"ID: {finetuning_id}")

In [ ]:
client = OpenAI()

training_data = [
    {"text": "Input text 1", "label": "Label 1"},
    {"text": "Input text 2", "label": "Label 2"},
    # 他のデータポイントも追加
]

response = openai.FineTune.create(
    model="text-davinci-003",  # ファインチューニングするモデルの指定
    training_data=training_data,
    epochs=5,  # エポック数の指定
    batch_size=4,  # バッチサイズの指定
    validation_data=None,  # バリデーションデータの指定
    prompt=None,  # プロンプトの指定
    start_from="latest_checkpoint",  # ファインチューニングを再開する場合に指定
)


In [4]:
completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": """
     You will be given a Japanese rakugo excerpt and a JLPT level.
     Rakugo excerpts will be covered by \"\"\".
     Do not output \"\"\".
     Please change the excerpt to use vocabulary and grammar appropriate for the given JLPT level.
     """},
    {"role": "user", "content": f"""
     The input is: \"\"\"{input_text}\"\"\".
     The JLPT level is: {JLPT_level}.
     """}
  ]
)

print(completion.choices[0].message)

ChatCompletionMessage(content='半端じゃないよ。これが嘘でも2円、3円、5円と集まってきてごらん。みんな言います。「2円、持ち合わせがなくて」と。また、そんなたくさんのお金だったら「もしかして返してくれないと困るな」と、断りたいし、断ることもできない。', role='assistant', function_call=None, tool_calls=None)


In [5]:
print(completion.choices[0].message.content)

半端じゃないよ。これが嘘でも2円、3円、5円と集まってきてごらん。みんな言います。「2円、持ち合わせがなくて」と。また、そんなたくさんのお金だったら「もしかして返してくれないと困るな」と、断りたいし、断ることもできない。
